In [2]:
// Import the API, Keyring and some utility functions
const { ApiPromise } = require('@polkadot/api');
const { Keyring } = require('@polkadot/keyring');

const BOB = '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty';

async function main () {
  // Instantiate the API
  const api = await ApiPromise.create();

  // Constuct the keying after the API (crypto has an async init)
  const keyring = new Keyring({ type: 'sr25519' });

  // Add Alice to our keyring with a hard-deived path (empty phrase, so uses dev)
  const alice = keyring.addFromUri('//Alice');

  // Create a extrinsic, transferring 12345 units to Bob
  const transfer = api.tx.balances.transfer(BOB, 12345);

  // Sign and send the transaction using our account
  const hash = await transfer.signAndSend(alice);

  console.log('Transfer sent with hash', hash.toHex());
}
await main()

Unknown types found, no types for Releases


Transfer sent with hash 0x67457f5503bf81f7c501706f3dd57f7952b97b5376b500035ecd09e0384b3f8c


In [3]:
// Import the API
const { ApiPromise } = require('@polkadot/api');

async function main () {
  // Create our API with a default connection to the local node
  const api = await ApiPromise.create();

  // Subscribe to system events via storage
  api.query.system.events((events) => {
    console.log(`\nReceived ${events.length} events:`);

    // Loop through the Vec<EventRecord>
    events.forEach((record) => {
      // Extract the phase, event and the event types
      const { event, phase } = record;
      const types = event.typeDef;

      // Show what we are busy with
      console.log(`\t${event.section}:${event.method}:: (phase=${phase.toString()})`);
      console.log(`\t\t${event.meta.documentation.toString()}`);

      // Loop through each of the parameters, displaying the type and data
      event.data.forEach((data, index) => {
        console.log(`\t\t\t${types[index].type}: ${data.toString()}`);
      });
    });
  });
}

main().catch((error) => {
  console.error(error);
//   process.exit(-1);
});

Promise { <pending> }


Unknown types found, no types for Releases



Received 2 events:
	system:ExtrinsicSuccess:: (phase={"ApplyExtrinsic":0})
		[ An extrinsic completed successfully.]
			DispatchInfo: {"weight":158000000,"class":"Mandatory","paysFee":"Yes"}
	system:ExtrinsicSuccess:: (phase={"ApplyExtrinsic":1})
		[ An extrinsic completed successfully.]
			DispatchInfo: {"weight":0,"class":"Mandatory","paysFee":"Yes"}


In [1]:
/// Transaction with events

// Import the API & Provider and some utility functions
const { ApiPromise } = require('@polkadot/api');

// Import the test keyring (already has dev keys for Alice, Bob, Charlie, Eve & Ferdie)
const testKeyring = require('@polkadot/keyring/testing');

// Utility function for random values
const { randomAsU8a } = require('@polkadot/util-crypto');

// Some constants we are using in this sample
const ALICE = '5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY';
const AMOUNT = 10000;

async function main () {
  // Create the API and wait until ready
  const api = await ApiPromise.create();

  // Create an instance of our testing keyring
  // If you're using ES6 module imports instead of require, just change this line to:
  // const keyring = testKeyring();
  const keyring = testKeyring.default();

  // Get the nonce for the admin key
  const { nonce } = await api.query.system.account(ALICE);

  // Find the actual keypair in the keyring
  const alicePair = keyring.getPair(ALICE);

  // Create a new random recipient
  const recipient = keyring.addFromSeed(randomAsU8a(32)).address;

  console.log('Sending', AMOUNT, 'from', alicePair.address, 'to', recipient, 'with nonce', nonce.toString());

  // Do the transfer and track the actual status
  api.tx.balances
    .transfer(recipient, AMOUNT)
    .signAndSend(alicePair, { nonce }, ({ events = [], status }) => {
      console.log('Transaction status:', status.type);

      if (status.isInBlock) {
        console.log('Included at block hash', status.asInBlock.toHex());
        console.log('Events:');

        events.forEach(({ event: { data, method, section }, phase }) => {
          console.log('\t', phase.toString(), `: ${section}.${method}`, data.toString());
        });
      } else if (status.isFinalized) {
        console.log('Finalized block hash', status.asFinalized.toHex());

        // process.exit(0);
      }
    });
}

main().catch(console.error);

Promise { <pending> }


Unknown types found, no types for Releases


Sending 10000 from 5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY to 5Gc7YcykLx8W2R8yS6fzuHWPk3VvJg5adk8FrFEmugp5vnEG with nonce 2
Transaction status: Ready
Transaction status: InBlock
Included at block hash 0x1bf3a356abef22f015fa217cd25d85ddee4e067e4a236dc1ec0ef7093605c5df
Events:
	 {"ApplyExtrinsic":2} : treasury.Deposit [1136255950151]
	 {"ApplyExtrinsic":2} : system.ExtrinsicFailed [{"Module":{"index":6,"error":4}},{"weight":195000000,"class":"Normal","paysFee":"Yes"}]
Transaction status: Finalized
Finalized block hash 0x1bf3a356abef22f015fa217cd25d85ddee4e067e4a236dc1ec0ef7093605c5df
